# GRO620 - Activité procédurale 2

Dans cette activité procédurale, nous allons poser les bases dufiltrage numérique d'images. Vous reconnaîtrez des éléments du filtrage numérique que vous avez vu en S4.

Pour chaque question impliqant de la programmation, commencez par discuter de la procédure à suivre pour résoudre le problème. Nous validerons l'approche en classe avant de se lancer dans l'implémentation.

In [ ]:
# Préambule

import numpy as np
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

## Si vous utilisez Google Colab, vous devez d'abord monter votre Google Drive
## où se trouve vos données. 
## Commentez les trois lignes suivantes en ajustant le chemin vers votre propre
## dossier :

# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/gro620-e21

## Pour retrouver le chemin depuis Jupyter, vous pouvez utiliser ceci :
# !ls /content/gdrive/MyDrive


## Caractéristiques de la lumière

### Q1.1

Dans cette image synthétique : 

![](https://upload.wikimedia.org/wikipedia/commons/c/cd/Specular_highlight.jpg)

(source: [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Specular_highlight.jpg))

**a)** Quelle(s) partie(s) correspondent à l'illumination diffuse et les reflets spéculaires ?

illumination diffuse: Toute la section coloré, la lumière qui n'est pas absorbé par l'objet

reflets spéculaires: points blancs, réflexion directe de la lumière

**b)** Quelle information est nécessaire pour déterminer les caractéristiques et emplacements exacts des sources de lumières dans cette image ? Répondez en utilisant des éléments de la *Bidirectional Reflectance Distribution Function* (BRDF).

Non, manque beaucoup d'information (type de lumière, autres sources, etc.). Seulement des approximations peuvent être fait.

## Encodage de l'image

Pour les questions suivantes, vous aurez probablement besoin de lire la documentation de cv2.imread et matplotlib.pyplot.imshow :

[imread](https://pythonexamples.org/python-opencv-read-image-cv2-imread/)

[imshow](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.imshow.html)

Le code suivant charge une image et l'affiche en ligne :

In [ ]:
img_color = cv2.imread("images_doc/proc1-q3-color.jpeg")
plt.imshow(img_color)

(source de l'image: [PixaBay, Pexels](https://www.pexels.com/photo/apartment-architecture-block-blue-534124/))

### Q2.1

**a)** Ouvrez directement l'image dans un autre logiciel (le fichier se trouve dans images_doc/proc1-q3-color.jpeg) et comparez le résultat. Que remarquez vous ?

Couleur inverse entre mauve et rouge, vert est inchangé

**b)** Affichez seulement le premier canal de couleurs de l'image. Pensez à analyser la composition de la matrice image que OpenCV vous retourne. Expliquez ensuite ce que vous voyez.

In [ ]:
plt.imshow(img_color)              # Image originale
plt.figure()                       # Permet d'afficher plus d'une image dans la même cellule
plt.imshow(img_color[:,:,1], cmap="gray") # Le paramètre cmap n'a pas d'effet sur une image à plus d'un canal.
plt.figure()

**c)** Transformez maintenant l'image pour que les couleurs correspondent à ce que vous voyez en dehors de Jupyter.

In [ ]:
img_correcte = img_color.copy() # Permet de forcer une copie de l'image.
                                # img_correcte = img_color aurait modifié l'originale.

(B,G,R) = cv2.split(img_color)
zeros = np.zeros(img_color.shape[:2], dtype="uint8")
img_correcte = cv2.merge([R, G, B])
img_R = cv2.merge([R, zeros, zeros])
img_G = cv2.merge([zeros, G, zeros])
img_B = cv2.merge([zeros, zeros, B])
plt.imshow(img_correcte) 
plt.figure()
plt.imshow(img_R)
plt.figure()
plt.imshow(img_G)
plt.figure()
plt.imshow(img_B)

###Autre solution###
#img_correcte[:,:,0] = img_color[:,:,2] #R to front
#img_correcte[:,:,2] = img_color[:,:,0] #B to back

### Q2.2

Soit cette couleur dans l'espace Y'CbCr (on suppose chaque valeur comme étant encodée sur 8 bits) :

$c = [100, 150, 150]$

Trouvez sa valeur équivalente dans l'espace RGB.

In [ ]:
c = np.array([100,150,150])
A = np.array([[0.299, 0.587, 0.114],
              [-0.168736, -0.331264, 0.5],
              [0.5, -0.418688, -0.081312]])
B = np.array([0, 128, 128])
c_rgb = np.dot(np.linalg.inv(A), (c - B))
print(c_rgb)
###Méthode 2###
imgYCC = np.zeros([1,1,3], dtype=np.uint8)
imgYCC[:] = c
img_rgb = cv2.cvtColor(imgYCC, cv2.COLOR_YCrCb2RGB)
print(img_rgb)

## Filtrage point à point

### Q3.1

Soit cette image (chargée par OpenCV et affichée par matplotlib): 

In [ ]:
img_q31_org = cv2.imread("images_doc/proc2-q1-dock.jpeg")
img_q31_rgb = cv2.cvtColor(img_q31_org, cv2.COLOR_BGR2RGB) # Équivalent de la question Q3.1.c de l'activité procédurale 1.
plt.imshow(img_q31_rgb)

(Source de l'image originale : [Vlada Karpovich, Pexels](https://www.pexels.com/photo/snow-wood-landscape-mountains-4450090/))

Cette fonction affiche l'histogramme des trois composantes de l'image :

In [ ]:
channels = ('r','g','b')
for i, col in enumerate(channels):
    hist = cv2.calcHist([img_q31_rgb], [i], None, [256], [0,256])
    plt.plot(hist,color = col)
    plt.xlim([0,256])

Ajustez la plage dynamique en luminosité de l'image pour qu'elle couvre l'ensemble des valeurs possibles.

In [ ]:
# NOTE: On convertit d'abord en float32 dans la plage [0,1] pour
# simplifier la manipulation des images avec des facteurs non-entiers.
# Matplotlib détecte ceci et affichera l'image correctement.
img_q31_f = np.float32(img_q31_rgb) / 255.0
max_i = img_q31_f.max()

img_q31_f /= max_i #x/max = 1/255
img_q31_out = img_q31_f.copy() # À remplacer !

plt.imshow(img_q31_out)

## Q3.2

Soit maintenant cette image :

In [ ]:
img_q32_org = cv2.imread("images_doc/proc2-q1-object.jpeg")
img_q32_rgb = cv2.cvtColor(img_q32_org, cv2.COLOR_BGR2RGB) # Équivalent de la question Q3.1.c de l'activité procédurale 1.
plt.imshow(img_q32_rgb)

Tentez de mettre en place un algorithme basé sur la luminosité permettant d'éliminer l'arrière-plan de cette image pour qu'il ne reste que l'objet en jaune sur un fond le plus noir possible.

In [ ]:
img_q32_filt = img_q32_rgb.copy()
W = img_q32_filt.shape[1] # NOTE: L'ordre des dimensions est Y puis X ("row-major")
H = img_q32_filt.shape[0]

for x in range(W):
    for y in range(H):
        if img_q32_filt[x,y].max() < 195:
            img_q32_filt[x,y] = [0,0,0]
         
plt.imshow(img_q32_filt)
plt.figure()


###AUTRE MÉTHODE###
ret, img = cv2.threshold(img_q32_filt,190,255, cv2.THRESH_BINARY)

plt.imshow(img)


## Filtrage linéaire

### Q4.1 

Soit l'image suivante ainsi que sa transformée de Fourier :

In [ ]:
img_q4_org  = cv2.imread("images_doc/proc2-q2-texture.jpeg")
img_q4_mono = np.float32(cv2.cvtColor(img_q4_org, cv2.COLOR_BGR2GRAY)) / 255.0

def get_fft_mag(img):
    img_fft = np.fft.fft2(img)
    img_fft = np.fft.fftshift(img_fft)
    img_fft = 20*np.log(np.abs(img_fft))
    return img_fft
    
img_q4_fft = get_fft_mag(img_q4_mono)

plt.subplot(121),plt.imshow(img_q4_mono, cmap="gray")
plt.subplot(122),plt.imshow(img_q4_fft)

(Source de l'image originale : [Hoang Le, Pexels](https://www.pexels.com/photo/black-and-white-black-and-white-pattern-rough-978462/)).

**a)** Filtrez cette image à l'aide d'une convolution de façon à ce que la valeur de chaque pixel soit la valeur moyenne de ses voisins dans un carré de 15x15.

In [ ]:
filter_kernel = np.ones((15,15))

filter_kernel = filter_kernel/sum(filter_kernel)

img_rst = cv2.filter2D(img_q4_mono,-1,filter_kernel)
img_rst_fft = get_fft_mag(img_rst)

plt.subplot(121),plt.imshow(img_rst, cmap="gray")
plt.subplot(122),plt.imshow(img_rst_fft)

**b)** Comparez le résultat avec celui de la fonction cv2.GaussianBlur() avec un noyau de convolution de la même taille.

In [ ]:
img_q4_blur = img_q4_mono.copy()
img_q4_blur = cv2.GaussianBlur(img_q4_mono,(15,15),0)
img_q4_blur_fft = get_fft_mag(img_q4_blur)

plt.subplot(121),plt.imshow(img_q4_blur, cmap="gray")
plt.subplot(122),plt.imshow(img_q4_blur_fft)

**c)** Comment expliquez-vous la différence ?

Mean: Accentue certain pixel et crée de l'aliasing (carré)

Gaussian: Évite l'aliasing

### Q4.2

Utilisez un filtre linéaire pour extraire les contours de l'image fournie en Q3.2.

In [ ]:
img_q32_contour = img_q32_rgb.copy()
img_q32_contour = cv2.Sobel(img_q32_rgb, -1, 1, 1)
plt.imshow(img_q32_contour)

## Filtrage non-linéaire

### Q5.1

Soit cette image :

In [ ]:
img_q5_org = cv2.imread("images_doc/proc2-q3-sand.png")
img_q5_rgb = cv2.cvtColor(img_q5_org, cv2.COLOR_BGR2RGB)
plt.imshow(img_q5_rgb)

Utilisez un filtre non-linéaire pour retirer les taches noires sur la figure.

In [ ]:
img_q5_fixed = img_q5_rgb.copy()

## Filtrage morphologique et chaîne de traitement

### Q6.1 

À partir de l'image de la question Q1.2, combinez les filtres vus plus tôt pour ne conserver que le contour de l'objet de la figure (donc sans bruit de fond).

In [ ]:
img_q32_clean = img_q32_rgb.copy()
img_q32_filter = cv2.GaussianBlur(img_q32_rgb,(15,15),0)

#img_q32_clean = cv2.Sobel(img_q32_filter, -1, 0, 1)
img_q32_clean = cv2.Canny(img_q32_filter, 150,250)

plt.imshow(img_q32_clean)
